In [1]:
from Bio import SeqIO, pairwise2
from Bio.Seq import translate
from Bio.pairwise2 import format_alignment
import os
import pandas as pd
import numpy as np

refs = [
    'batch1_20220311',
    'batch2_20220413'
]

wt_genome = './SARS-CoV-2-spike.fasta'
for record in SeqIO.parse(wt_genome, 'fasta'):
    wt_aa = translate(record.seq.upper())

all_data_meta = []

for date in refs:
    summary = pd.read_csv(
        './'+date+'/summary.csv', 
        index_col=None
    ).fillna('')
    all_data_meta.append(summary)

all_data_meta = pd.concat(all_data_meta)


In [2]:
numbering = {}
genomes = {}

for genome in np.unique(all_data_meta['reference'].to_list()):
    print(genome)
    ref_genome = './genome/'+genome+'/'+genome+'.fasta'

    for record in SeqIO.parse(ref_genome, 'fasta'):
        _number = [0]
        alignments = pairwise2.align.globalxx(translate(record.seq.upper()), wt_aa)

        query, ref = alignments[0][0], alignments[0][1]
        _x = 1
        for i in range(1, len(ref)):
            if query[i] == ref[i]:
                _number.append(_x)
                _x += 1
            elif query[i] == '-':
                _x += 1
                continue
            elif ref[i] == '-':
                _number.append(_x)
            else:
                raise ValueError
        numbering[genome] = _number
        genomes[genome] = record.seq.upper()
        break # only the first record


rVSVOmicron


In [3]:
all_data_meta['file'] = ['' for i in range(len(all_data_meta))]
all_data_meta['date'] = all_data_meta['date'].astype(str)

nt_df = []
for (_i, row) in all_data_meta.iterrows():
    results = './'+row['date']+'/Snps'
    f = os.path.join(results,
                     row['sampleName'] + '.' + row['reference']+'.variants_dup_clean.mut.txt')
    if os.path.exists(f):
        row['file'] = f
    else:
        print('Not Found ' + f)
        continue
    try:
        _ = pd.read_csv(f,sep = "[ \t]",header = None,engine='python')
        _.columns = ['chr', 'pos', 'ref', 'depth', 'A', 'T', 'C', 'G', 'mut', 'mut_depth']
        _['name'] = row['sampleName']
        assert(len(_)>0)
        _.index = range(len(_))
        
    except:
        print("Error Reading: "+f)
        continue
    _['description'] = row['date']+'|'+str(row['abName'])+'|'+row['description']+'|'+row['reference']
    nt_df.append(_)
nt_df = (
    pd.concat(nt_df).reset_index().drop(columns=['index'])
      .melt(id_vars=['chr', 'pos', 'ref', 'depth', 'mut', 'mut_depth', 'name', 'description'])
      .query('value > 0')
      .reset_index()
      .drop(columns=['index'])
)
nt_df

,chr,pos,ref,depth,mut,mut_depth,name,description,variable,value
0,rVSVOmicron,7,G,9791,0.000238,2,PD10_XXL_YF0228-10,batch1_20220311|5840|P1-rep2|rVSVOmicron,A,0.000238
1,rVSVOmicron,10,T,12018,0.000098,1,PD10_XXL_YF0228-10,batch1_20220311|5840|P1-rep2|rVSVOmicron,A,0.000098
2,rVSVOmicron,16,G,16443,0.000550,9,PD10_XXL_YF0228-10,batch1_20220311|5840|P1-rep2|rVSVOmicron,A,0.000346
3,rVSVOmicron,18,G,18206,0.000367,7,PD10_XXL_YF0228-10,batch1_20220311|5840|P1-rep2|rVSVOmicron,A,0.000184
4,rVSVOmicron,20,T,19672,0.000394,8,PD10_XXL_YF0228-10,batch1_20220311|5840|P1-rep2|rVSVOmicron,A,0.000338
...,...,...,...,...,...,...,...,...,...,...
499745,rVSVOmicron,3728,G,30324,0.000516,16,PD10_XXL_0413_YF_lib_46_54-9,batch2_20220413|5514|P4-rep2|rVSVOmicron,G,0.999484
499746,rVSVOmicron,3729,C,29260,0.000692,20,PD10_XXL_0413_YF_lib_46_54-9,batch2_20220413|5514|P4-rep2|rVSVOmicron,G,0.000154
499747,rVSVOmicron,3731,G,26963,0.000396,11,PD10_XXL_0413_YF_lib_46_54-9,batch2_20220413|5514|P4-rep2|rVSVOmicron,G,0.999604
499748,rVSVOmicron,3734,G,23599,0.000237,6,PD10_XXL_0413_YF_lib_46_54-9,batch2_20220413|5514|P4-rep2|rVSVOmicron,G,0.999763


In [4]:
CUTOFF = 0.001

use_df = nt_df.query('chr == "rVSVOmicron" and value > @CUTOFF and ref != variable').copy()
use_df.index = range(len(use_df))
use_df

,chr,pos,ref,depth,mut,mut_depth,name,description,variable,value
0,rVSVOmicron,71,T,60134,0.001268,76,PD10_XXL_YF0228-10,batch1_20220311|5840|P1-rep2|rVSVOmicron,A,0.001157
1,rVSVOmicron,374,T,164663,0.001489,245,PD10_XXL_YF0228-10,batch1_20220311|5840|P1-rep2|rVSVOmicron,A,0.001193
2,rVSVOmicron,449,C,196138,0.006914,1356,PD10_XXL_YF0228-10,batch1_20220311|5840|P1-rep2|rVSVOmicron,A,0.006739
3,rVSVOmicron,460,G,198762,0.002232,444,PD10_XXL_YF0228-10,batch1_20220311|5840|P1-rep2|rVSVOmicron,A,0.002222
4,rVSVOmicron,855,C,190053,0.031906,6064,PD10_XXL_YF0228-10,batch1_20220311|5840|P1-rep2|rVSVOmicron,A,0.031224
...,...,...,...,...,...,...,...,...,...,...
5380,rVSVOmicron,3424,T,127376,0.006642,846,PD10_XXL_0413_YF_lib_46_54-9,batch2_20220413|5514|P4-rep2|rVSVOmicron,G,0.006555
5381,rVSVOmicron,3516,T,156390,0.002346,367,PD10_XXL_0413_YF_lib_46_54-9,batch2_20220413|5514|P4-rep2|rVSVOmicron,G,0.002090
5382,rVSVOmicron,3592,C,164619,0.022918,3773,PD10_XXL_0413_YF_lib_46_54-9,batch2_20220413|5514|P4-rep2|rVSVOmicron,G,0.019175
5383,rVSVOmicron,3613,C,162857,0.008534,1390,PD10_XXL_0413_YF_lib_46_54-9,batch2_20220413|5514|P4-rep2|rVSVOmicron,G,0.004013


In [5]:
# add aa site, wt, mut

show_names = {
    '5514': 'SA55',
    '5840': 'SA58',
    '5840+5514': 'SA55+SA58',
}

for _i in use_df.index:
    row = use_df.loc[_i,:]
    ch, pos = row['chr'], row['pos']-1
    aa_pos = pos // 3
    st = aa_pos * 3
    ed = st + 3
    
    rel_pos = pos % 3
    ref_codon = str(genomes[ch][st:ed])
    ref_aa = translate(ref_codon)

    use_df.loc[_i,'wt_aa'] = ref_aa

    if use_df.loc[_i,'variable'] == row['ref']:
        use_df.loc[_i,'mut_aa'] == ref_aa
    else:
        mut_codon = ref_codon[0:rel_pos] + row['variable'] + ref_codon[(rel_pos+1):]
        use_df.loc[_i,'mut_aa'] = translate(mut_codon)
    use_df.loc[_i,'site'] = numbering[ch][aa_pos]+1
    
    ds = row['description'].split('|')
    use_df.loc[_i, 'antibody'] = show_names[ds[1]]
    
    _ep = ds[2].split('-')
    use_df.loc[_i, 'ep'] = _ep[0]
    if len(_ep) > 1:
        use_df.loc[_i, 'rep'] = _ep[1]
    else:
        use_df.loc[_i, 'rep'] = 'repUKN'
    
    if (_i % 1000 == 0):
        print("Processed", _i)


Processed 0
Processed 1000
Processed 2000
Processed 3000
Processed 4000
Processed 5000


In [6]:
output_df = use_df.assign(site_mut = use_df['wt_aa']+use_df['site'].astype(int).astype(str)+use_df['mut_aa'])
output_df.to_csv("rVSV_results.csv", index=None)